# Replication - Habitat, No-take MPAs 2.4

### Import libraries and set up environment

In [2]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [3]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [4]:
# Intersect the dissolved no-take habitat MPA layer with the habitat layer

in_feature_1 = "WDPA_MPA_30km_NoTake_dis"
in_feature_2 = "habitats_dis_merge"
out_feature = "\\MPA_30km_NoTake_habitat_int_Replic2_4"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseIntersect(
    in_features=[in_feature_1,in_feature_2],
    out_feature_class=output_gdb + out_feature,
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MPA_30km_NoTake_habitat_int_Replic2_4'>

In [5]:
# Repair geoemtry on intersect and calculate area of intersecting bits

in_feature = "MPA_30km_NoTake_habitat_int_Replic2_4"
out_feature = "MPA_30km_NoTake_habitat_int_singl_Replic2_4"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def repair_geo_area():
    #repair geometry
    arcpy.management.RepairGeometry(
        in_features= in_feature,
        delete_null="DELETE_NULL",
        validation_method="OGC"
    )
    
    #make the intersected layer signlepart to ensure all polygons are individual
    arcpy.management.MultipartToSinglepart(
        in_features= in_feature,
        out_feature_class=out_feature
    )
            
    # insert "area" field (double) and calculate area in km2 in Mollweide
    arcpy.management.CalculateGeometryAttributes(
        in_features= out_feature,
        geometry_property="area AREA",
        length_unit="",
        area_unit="SQUARE_KILOMETERS",
        coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
        coordinate_format="SAME_AS_INPUT"
    )

repair_geo_area()


In [6]:
# Only keep features with an area greater than .24 km2

# Select features with an area greater than .24 km2 and make a new layer from that and export it to excel

# Set a folder to store the output tables
table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Replication"

in_feature = "MPA_30km_NoTake_habitat_int_singl_Replic2_4"
out_feature = "MPA_30km_NoTake_habitat_int_singl_big_Replic2_4"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def select_big_excel():
    # make a layer of polygons with an area greater than .24 km2    
    arcpy.conversion.ExportFeatures(
        in_features= in_feature,
        out_features= out_feature,
        where_clause="area >= 0.24",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=None,
        sort_field=None
    )
    
    # Export table to excel
    # Not sure why I had to use the following two lines instead of just using the TableToExcel expression, but I did
    excel_filename = out_feature + ".xlsx"
    excel_filepath = os.path.join(table_outputs_folder, excel_filename)  # Build the full file path

    arcpy.conversion.TableToExcel(
        Input_Table= in_feature,
        Output_Excel_File= excel_filepath,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )
    
select_big_excel()

In [7]:
# Convert the output features to a Pandas DataFrame
df = pd.DataFrame.spatial.from_featureclass("MPA_30km_NoTake_habitat_int_singl_big_Replic2_4")


In [8]:
# Display the DataFrame
df.head()

,OBJECTID,FID_WDPA_MPA_30km_NoTake_dis,FID_habitats_dis_merge,Id,gridcode,Habitat,ORIG_FID,area,SHAPE
0,1,22,687,2755.0,1.0,shelf_hard,1,2.619752,"{""rings"": [[[-6176885.852499999, 2176878.55629..."
1,2,22,700,2771.0,1.0,shelf_hard,2,113.522600,"{""rings"": [[[-6180623.767999999, 2158188.97870..."
2,3,22,702,2774.0,1.0,shelf_hard,3,0.873251,"{""rings"": [[[-6180623.767999999, 2158188.97870..."
3,4,22,738,2823.0,1.0,shelf_hard,4,5.239505,"{""rings"": [[[-6192771.9934, 2120809.8236999996..."
4,5,22,742,2830.0,1.0,shelf_hard,5,70.733313,"{""rings"": [[[-6192771.9934, 2120809.8236999996..."


In [9]:
# Make pivot tables
pivot_MPA_ecoregion = df.pivot_table(
     index = "Habitat",
     values = "OBJECTID",
     aggfunc = "count"
).reset_index()
pivot_MPA_ecoregion.columns = ["Habitat", "count"]
print(pivot_MPA_ecoregion)



           Habitat  count
0            coral    286
1        deep_hard     16
2        deep_soft     13
3         mangrove    244
4        saltmarsh    189
5         seagrass    183
6   seamount_knoll     20
7     shallow_hard    109
8     shallow_soft    159
9       shelf_hard    134
10      shelf_soft     84
11      slope_hard     25
12      slope_soft     37


In [10]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Replication"
table = pivot_MPA_ecoregion
output_name = "MPA_30km_NoTake_habitat_int_singl_big_Replic2_4_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))

Reminder to add in the marine ecoregions/pelagic provinces that fall within the study area but don't have any intersections with no-take MPAs so they dont show up in the results.